# Perform Calculations on Intervals

## Creating the client

In [1]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net"
client_id = "tmInterfaceClient"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    tz = "Europe/Brussels"
)

# Show client
client

<< TrendMinerClient | https://cs.trendminer.net | service-account-tminterfaceclient >>

## Selecting tags

In [2]:
level, conc, temp, phase = client.tag.list(
    ["[CS]BA:LEVEL.1", 
     "[CS]BA:CONC.1", 
     "[CS]BA:TEMP.1", 
     "[CS]BA:PHASE.1"
    ])

# Show level as example
level

<< Tag | [CS]BA:LEVEL.1  | 0:00:00 >>

## Selecting intervals
Intervals can come from many sources, such as search results, context items, the range of a view... For this example, let's generate intervals directly.

In [3]:
# All full weeks (starting Monday) since the start of 2022, in client timezone
weeks = client.time.interval.range(
    start="2022-01-01 00:00:00",
    end=client.time.now(),
    freq="W-MON")

# Show first 4 weeks as example
weeks[0:4]

[<< Interval | 2022-01-03 00:00:00+01:00 | 7 days, 0:00:00 >>,
 << Interval | 2022-01-10 00:00:00+01:00 | 7 days, 0:00:00 >>,
 << Interval | 2022-01-17 00:00:00+01:00 | 7 days, 0:00:00 >>,
 << Interval | 2022-01-24 00:00:00+01:00 | 7 days, 0:00:00 >>]

## Performing the calculations
Calculations are implemented as Tag methods. Performing a calculation returns a list of intervals with the calculation added to the interval.

In [4]:
# Add the average temperature
weeks_w_temp = temp.calculate(intervals=weeks, operation="mean", key="temp")

# Show first week as example
print(weeks_w_temp[0])

# Get value only
print(weeks_w_temp[0]["temp"])

<< Interval | 2022-01-03 00:00:00+01:00 | 7 days, 0:00:00  | temp: 18.277672>>
18.277672


We can also modify the original intervals in place. Let's perform a number of calculations on the original weeks. For tags with string values, we can get the start and end values without issues.

In [5]:
temp.calculate(intervals=weeks, operation="mean", key="avg temp", inplace=True)
temp.calculate(intervals=weeks, operation="std", key="temp std", inplace=True)
conc.calculate(intervals=weeks, operation="start", key="start conc", inplace=True)
level.calculate(intervals=weeks, operation="start", key="start level", inplace=True)
phase.calculate(intervals=weeks, operation="start", key="start phase", inplace=True)

# print first week as example
print(weeks[0])

<< Interval | 2022-01-03 00:00:00+01:00 | 7 days, 0:00:00  | avg temp: 18.277672, temp std: 12.794154, start conc: 8.120249, start level: 17.849695, start phase: Phase1>>


## Custom calculations
You can add any value into an interval, allowing you to add custom calculations. An interval can be treated like a dict in this respect.

In [6]:
tank_area = 7.42

for week in weeks:
    week["total product start"] = week["start conc"] * week["start level"] * tank_area

# Print result for first week
print(weeks[0]["total product start"])

1075.4842423674882


## To DataFrame
It can make sense to represent results as a pandas DataFrame. There are a number of ways to achieve this, of which ones is as follows:

In [7]:
import pandas as pd

# Values to dataframe
df = pd.DataFrame.from_records(weeks)

# Start/end timestamps and duration
df_time = pd.DataFrame.from_records(
    [{"start": week.start, "end": week.end, "duration": week.total_seconds()} for week in weeks]
)

df = pd.concat([df_time, df], axis=1)

# Display first rows as example
df.head()

start                       end  duration   avg temp  \
0 2022-01-03 00:00:00+01:00 2022-01-10 00:00:00+01:00  604800.0  18.277672   
1 2022-01-10 00:00:00+01:00 2022-01-17 00:00:00+01:00  604800.0  18.414118   
2 2022-01-17 00:00:00+01:00 2022-01-24 00:00:00+01:00  604800.0  18.202066   
3 2022-01-24 00:00:00+01:00 2022-01-31 00:00:00+01:00  604800.0  18.500250   
4 2022-01-31 00:00:00+01:00 2022-02-07 00:00:00+01:00  604800.0  18.381855   

   start conc  start level start phase   temp std  total product start  
0    8.120249    17.849695      Phase1  12.794154          1075.484242  
1   26.640080    44.092754      Phase4  12.884811          8715.787945  
2    0.000000     0.000000      Phase1  12.780355             0.000000  
3   19.147213    34.390106      Phase3  12.901333          4885.882160  
4   44.520275     3.672416      Phase7  12.795521          1213.147585